In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))
print(f"Project root added to sys.path: {project_root}")

from src.devices.vogtlin_mfc import VogtlinMFC

Project root added to sys.path: /home/daniel/repos/lab-controller


In [3]:
dry_mfc = VogtlinMFC(port="/dev/ttyS4", baudrate=9600, address=11)

if dry_mfc.connect():
    dry_mfc.get_status()
    dry_mfc.set_flow_setpoint(0.5)
    dry_mfc.get_status()
    dry_mfc.set_flow_setpoint(0)

Error connecting to MFC on /dev/ttyS4: [Errno 5] could not open port /dev/ttyS4: [Errno 5] Input/output error: '/dev/ttyS4'


In [4]:
from src.devices.dewmaster import DewMasterHygrometer

hygrometer = DewMasterHygrometer(port="/dev/ttyS3", baudrate=9600)
hygrometer.connect()
t = hygrometer.get_ambient_temperature()
print(t)

TAMB


In [26]:
import serial

# Adjust to your system’s COM port or /dev/ttyUSBx
port = "/dev/ttyS5"  # or "/dev/ttyUSB0" on Linux

ser = serial.Serial(
    port=port,
    baudrate=19200,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1,
    xonxoff=True
)

print("Connected to DewMaster on", port)
print("Reading measurements...")

try:
    while True:
        line = ser.readline().decode(errors="ignore").strip()
        print(line)
        if line:
            print(line)
            # Optional: parse values
            # Example if line looks like "T=23.45C RH=45.67%"
            parts = line.split()
            data = {}
            for p in parts:
                if "=" in p:
                    k, v = p.split("=")
                    data[k] = v.strip("%C")
            if "T" in data and "RH" in data:
                print(f"Temperature: {data['T']} °C, Humidity: {data['RH']} %")
except KeyboardInterrupt:
    ser.close()
    print("Connection closed.")


Connected to DewMaster on /dev/ttyS5
Reading measurements...










































Connection closed.


In [29]:
import serial
import time

PORT = "/dev/ttyS5"   # Replace with your serial port
BAUDRATE = 19200

with serial.Serial(PORT, BAUDRATE, timeout=1, xonxoff=True) as ser:
    time.sleep(1)
    ser.reset_input_buffer()
    print(f"Connected to DewMaster on {PORT} @ {BAUDRATE} baud...")
    print("Monitoring (Ctrl+C to stop)...\n")

    try:
        while True:
            # Send command to request a reading
            ser.write(b"READ\r\n")
            time.sleep(0.5)  # give the device a moment to respond

            # Read all available lines from the device
            while ser.in_waiting:
                line = ser.readline().decode(errors="ignore").strip()
                if line:
                    print(line)

            time.sleep(1)

    except KeyboardInterrupt:
        print("\nStopped by user.")



Connected to DewMaster on /dev/ttyS5 @ 19200 baud...
Monitoring (Ctrl+C to stop)...

R
R
R
R
READ
11/13/25  13:15:51   FP =   -4.3 C  AT  =   24.0 C  RH  =   14.4    SERVOLOCK
11/13/25  13:16:19   FP =   -4.2 C  AT  =   24.0 C  RH  =   14.4    SERVOLOCK
Input:  R
R
R
READ
Invalid command:  RRRREAD
11/13/25  13:16:19   FP =   -4.2 C  AT  =   24.0 C  RH  =   14.4    SERVOLOCK
11/13/25  13:16:28   FP =   -4.2 C  AT  =   24.0 C  RH  =   14.5    SERVOLOCK
Input:  R
R
R
READ
Invalid command:  RRRREAD
11/13/25  13:16:28   FP =   -4.1 C  AT  =   24.0 C  RH  =   14.6    SERVOLOCK
11/13/25  13:16:37   FP =   -4.1 C  AT  =   24.0 C  RH  =   14.6    SERVOLOCK
Input:  R

Stopped by user.


In [34]:
import serial
import time

PORT = "/dev/ttyS5"
BAUDRATE = 19200

with serial.Serial(PORT, BAUDRATE, timeout=1, xonxoff=True) as ser:
    time.sleep(1)
    ser.reset_input_buffer()
    print(f"Connected to DewMaster on {PORT} @ {BAUDRATE} baud...")

    try:
        while True:
            # Flush buffer to remove echoes
            ser.reset_input_buffer()
            
            # Send READ command
            ser.write(b"ST\r\n")
            time.sleep(1)  # wait for full response
            
            # Read all available lines
            while ser.in_waiting:
                line = ser.readline().decode(errors="ignore").strip()
                if line:
                    print(line)
            
            time.sleep(5)  # wait before next command

    except KeyboardInterrupt:
        print("\nStopped by user.")


Connected to DewMaster on /dev/ttyS5 @ 19200 baud...
ST
11/13/25  13:20:33   FP =   -2.5 C  AT  =   24.0 C  RH  =   16.6    SERVOLOCK
11/13/25  13:21:33   FP =   -2.5 C  AT  =   24.0 C  RH  =   16.6    SERVOLOCK
Input:  S
S
S
S
S
S
S
S
S
S
S
S
S
ST
Invalid command:  SSSSSSSSSSSSSST
11/13/25  13:21:33   FP =   -1.9 C  AT  =   24.0 C  RH  =   17.4    SERVOLOCK
11/13/25  13:23:10   FP =   -1.9 C  AT  =   24.0 C  RH  =   17.4    SERVOLOCK
Input:  S
S
S
S
S
S
S
S
S

Stopped by user.


In [56]:
import serial
import time

# --- Serial port configuration ---
PORT = "/dev/ttyS5"          # e.g., /dev/ttyUSB0 on Linux
BAUD_RATE = 9600
TIMEOUT = 2

TIMEOUT = 2  # Timeout in seconds

def main():
    try:
        # Open serial connection
        ser = serial.Serial(
            port=PORT,
            baudrate=BAUD_RATE,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=TIMEOUT
        )
        
        print(f"Connected to DewMaster on {PORT} at {BAUD_RATE} baud")
        print("Reading data every second. Press Ctrl+C to stop.\n")
        
        # Clear any existing data in buffer
        ser.reset_input_buffer()
        
        while True:
            # Send poll command to request data
            ser.write(b'P\r')
            
            # Wait a moment for response
            time.sleep(0.5)
            
            # Read response
            if ser.in_waiting > 0:
                response = ser.read(ser.in_waiting).decode('ascii', errors='ignore')
                print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - {response.strip()}")
            
            # Wait before next reading
            # time.sleep(1)
            
    except serial.SerialException as e:
        print(f"Serial port error: {e}")
    except KeyboardInterrupt:
        print("\nStopped by user")
    finally:
        if 'ser' in locals() and ser.is_open:
            ser.close()
            print("Serial port closed")

if __name__ == "__main__":
    main()


Connected to DewMaster on /dev/ttyS5 at 9600 baud
Reading data every second. Press Ctrl+C to stop.

2025-11-13 14:43:12 - Input:  P
2025-11-13 14:43:12 - P
2025-11-13 14:43:13 - P
2025-11-13 14:43:13 - P
2025-11-13 14:43:14 - P
2025-11-13 14:43:14 - P
2025-11-13 14:43:15 - P
2025-11-13 14:43:15 - P
2025-11-13 14:43:16 - P
2025-11-13 14:43:16 - P
2025-11-13 14:43:17 - P
2025-11-13 14:43:17 - P
2025-11-13 14:43:18 - P
2025-11-13 14:43:18 - P

Stopped by user
Serial port closed
